In [186]:
#######################################################################
# Libraries
#######################################################################

In [187]:
# Libraries
import pandas as pd
import numpy as np
import reverse_geocoder as rg

In [188]:
#######################################################################
# Functions
#######################################################################

In [189]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

In [190]:
# Function Reverse Geocoding
def getPlace(location):
    # Create latList and lonList
    latList = df.lat[df.location == location].tolist()
    lonList = df.long[df.location == location].tolist()
    
    # Clean latList, lonList
    def cleanList(dirtyList):
        index = 0
        while(index < len(dirtyList)):
            if np.isnan(dirtyList[index]):
                del dirtyList[index]
            else:
                index += 1
        return dirtyList
    
    latList = cleanList(latList)
    lonList = cleanList(lonList)
    
    # Find median lat lon
    if 0 != len(latList) or 0 != len(lonList):
        lat = np.median(latList)
        lon = np.median(lonList)
    else:
        return['NaN', 'NaN']
    
    # Reverse Geocode form median lat lon
    try:
        result = rg.search((lat, lon))
    except IndexError:
        return ["ERROR: " + location, "ERROR: " + location]
    
    # Format output    
    return [result[0]['name'], us_state_abbrev[result[0]['admin1']]]

In [191]:
# Function finds the spelling mistake with a levenshtein difference of one
def fixSpelling(word, possibleWords):
    alphabet = '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes    = [a + b[1:] for a, b in splits if b]
    transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
    replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
    inserts    = [a + c + b     for a, b in splits for c in alphabet]
   
    allWords = set(deletes + transposes + replaces + inserts)

    for element in allWords:
        if element in possibleWords:
            return element
        
    return "NaN"

In [192]:
#######################################################################
# Load Data
#######################################################################

In [193]:
# Load Data
df = pd.read_csv('resources/data/CarSale_Data_Raw.csv')

In [194]:
#######################################################################
#  General Cleaning
#######################################################################

In [195]:
# Remove all NaN
df.dropna(inplace = True)

In [196]:
# Remove Columns
df.drop(["url", "city_url", "VIN", "image_url", "desc"], axis=1, inplace = True)

In [197]:
# Rename
df.rename(columns = {"make":"model", "manufacturer":"make", "city":"location", "paint_color":"color", "title_status":"title"}, inplace=True);

In [198]:
# Cast Type
df = df.astype({'year':int, 'odometer':int});

In [199]:
#######################################################################
#  Rough Cuts
#######################################################################

In [200]:
# Year
df = df[(df.year >= 2000) & (df.year != 2020)]

In [201]:
# Price
df = df[df.price <= df.price.quantile(.95)]

In [202]:
# Odometer
#df = df[df.odometer <= df.odometer.quantile(.95)]

In [203]:
# Reduce Files Size
df = df.sample(10000)

In [204]:
#######################################################################
# Locations
#######################################################################

In [205]:
# Add City, State Column
df.insert(0, "city", "NaN") # Add new columns for city
df.insert(1, "state", "NaN") # Add new column for state

In [206]:
# Find unique locations
uniqueLocations = df["location"].unique() 

# Create and Build dictionary
loc2CityState = {}

for location in uniqueLocations:
    if "," in location:    
        loc2CityState[location] = location.split(", ")
    elif "/" in location:
        cityCity = location.split("/")
        state = getPlace(location)[1]
        loc2CityState[location] = [cityCity[0], state]
    elif "-" in location:
        cityCityCity = location.split("-")
        state = getPlace(location)[1]
        loc2CityState[location] = [cityCityCity[0], state]
    elif location.title() in us_state_abbrev:
        city = getPlace(location)[0]
        loc2CityState[location] = [city, us_state_abbrev[location.title()]]
    else:
        state = getPlace(location)[1]
        loc2CityState[location] = [location, state]
        
# Clean location into City State
for i in df.index:    
    location = df.at[i, 'location']
    cityState = loc2CityState[location]
    
    df.at[i, 'city'] = cityState[0]
    df.at[i, "state"] = cityState[1]

In [207]:
# Drop pointless columns
df.drop(['location'], axis=1, inplace=True)

In [208]:
#######################################################################
# Makes
#######################################################################

In [209]:
# Grab makes
urlmake = "https://vpic.nhtsa.dot.gov/api/vehicles/GetAllMakes?format=csv"
makeFullList = pd.read_csv(urlmake)['make_name'].tolist()
makeFullList = [x.lower() for x in makeFullList]

In [210]:
# Edit names
df['make'].replace('-', ' ', inplace=True)

In [211]:
# Spell Check Names
makeDict = {}
makeList = df['make'].unique().tolist()

for word in makeList:
    if word in makeFullList:
        makeDict[word] = word 
    else:
        makeDict[word] = fixSpelling(word, makeFullList) 
        
df['make'].replace(makeDict, inplace=True)

In [216]:
#######################################################################
# Save
#######################################################################

In [217]:
# Save
df.to_csv('resources/data/CarSale_Data_Rough.csv', index=False)